The nv center paper https://pubs.aip.org/avs/aqs/article/2/2/024701/997281/Introduction-to-quantum-optimal-control-for has an example of a control pulse for just the zeeman term of the Hamiltonian.  I want to replicate this and put it in GRAPE to try to understand how to deal with the B-field being aligned or not aligned with the NV-axes.

In [2]:
import matplotlib.pyplot as plt
import time
import numpy as np
from numpy import pi

from scipy.constants import physical_constants, h
from dataclasses import dataclass

from qutip import *
from qutip.control import *
from qutip.qip.operations import rx
from qutip.qip.operations import ry
from qutip.qip.operations import rz

from qutip.control.grape import plot_grape_control_fields, _overlap
from qutip.control.cy_grape import cy_overlap
from qutip.control.grape import cy_grape_unitary, grape_unitary_adaptive

from scipy.interpolate import interp1d
from qutip.ui.progressbar import TextProgressBar

In [3]:
'''def rz3(phi, N=None, target=0):
    
    rotationx=Qobj([[np.exp(-1j*(phi/2)),0,0],
                            [0,0,0],
                            [0,0,-np.exp(1j*(phi/2))]])
    
    return rotationx


T = 1
times = np.linspace(0, T, 100)

#theta, phi = np.random.rand(2)
theta, phi = [pi/2, pi/4]

# target unitary transformation (random single qubit rotation)
#U = rz(phi) * rx(theta) 
#U

U= rz3(phi) #* rx3(theta) 
U'''

'def rz3(phi, N=None, target=0):\n    \n    rotationx=Qobj([[np.exp(-1j*(phi/2)),0,0],\n                            [0,0,0],\n                            [0,0,-np.exp(1j*(phi/2))]])\n    \n    return rotationx\n\n\nT = 1\ntimes = np.linspace(0, T, 100)\n\n#theta, phi = np.random.rand(2)\ntheta, phi = [pi/2, pi/4]\n\n# target unitary transformation (random single qubit rotation)\n#U = rz(phi) * rx(theta) \n#U\n\nU= rz3(phi) #* rx3(theta) \nU'

ok now I'm going to write out the full Hamiltonian for that term

In [8]:
nuclear_spin = 1
electron_spin = 1
sigmaxE = jmat(electron_spin,'x')
sigmayE = jmat(electron_spin,'y')
sigmazE = jmat(electron_spin,'z')
sigmaxN = jmat(nuclear_spin,'x')
sigmayN = jmat(nuclear_spin,'y')
sigmazN = jmat(nuclear_spin,'z')
sigmazEsquared = sigmazE*sigmazE
sigmazNsquared = sigmazN*sigmazN
identity3 = identity(3) #3x3 identity matrix



#Creating operators that can be tuned in the Hamiltonian via the control fields, list is below called H_ops

H_opsEx=tensor(sigmaxE, identity3)
H_opsEy=tensor(sigmayE, identity3)
H_opsEz=tensor(sigmazE, identity3)
H_opsNx=tensor(identity3, sigmaxN)
H_opsNy=tensor(identity3, sigmayN)
H_opsNz=tensor(identity3, sigmazN)

H_opsENx = tensor(sigmaxE, sigmaxN)
H_opsENy = tensor(sigmayE, sigmayN)           
H_opsENz = tensor(sigmazE, sigmazN)
print (H_opsENz)

#changing dims bc qutip is silly

H_opsEx.dims=[[9],[9]]
H_opsEy.dims=[[9],[9]]
H_opsEz.dims=[[9],[9]]

H_opsNx.dims=[[9],[9]]
H_opsNy.dims=[[9],[9]]
H_opsNz.dims=[[9],[9]]

H_opsENx.dims=[[9],[9]]
H_opsENy.dims=[[9],[9]]
H_opsENz.dims=[[9],[9]] 
#making H_ops list           
H_ops = [H_opsEx, H_opsEy, H_opsEz, H_opsNx, H_opsNy, H_opsNz, H_opsENx, H_opsENy, H_opsENz]            
               
               
#making labels for pulse sequence graphs
H_labels = [r'$uE_{x}$',
            r'$uE_{y}$',
            r'$uE_{z}$',
            r'$uN_{x}$',
            r'$uN_{y}$',
            r'$uN_{z}$',
            r'$uEN_{x}$',
            r'$uEN_{y}$',
            r'$uEN_{z}$']

Quantum object: dims = [[3, 3], [3, 3]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]]


In [12]:
#defining hamiltonian for NV center

#constants

f_fine_structure = 2.87e9
f_nuclear_quadrupole = -5.01e6
f_axial_magnetic_hyperfine = -2.14e6
f_transverse_magnetic_hyperfine = -2.7e6
g_factor_electron = 2.0028
gyromagnetic_constant_nuclear = 1.93297e7 / (2 * np.pi)
uB = physical_constants['Bohr magneton'][0]
uN = physical_constants['nuclear magneton'][0]    
bvector=[0,0,1]

H5 = ((uB/h) * g_factor_electron) * tensor((sigmaxE*bvector[0] + sigmayE*bvector[1] + sigmazE*bvector[2]), identity3)

H6 = gyromagnetic_constant_nuclear * tensor(identity3,(sigmaxN*bvector[0] + sigmayN*bvector[1] + sigmazN*bvector[2]))

    
H0= H5
H0.dims=[[9],[9]] #changing dimensions bc dot products are silly
#H0 #hamiltonian without nuclear b field term
H5

Quantum object: dims = [[9], [9]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[ 2.80316794e+10  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  2.80316794e+10  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  2.80316794e+10  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0

In [16]:
U = 0 * H5

In [17]:
R = 150 #GRAPE itterations

T = 1
times = np.linspace(0, T, 100)
##creating initial control pulse values as starting guess for GRAPE

u0 = np.array([np.random.rand(len(times)) * 2 * pi * 0.005 for _ in range(len(H_ops))])

u0 = [np.convolve(np.ones(10) / 10, u0[idx, :], mode='same') for idx in range(len(H_ops))]

In [18]:
#doing GRAPE!

result = cy_grape_unitary(U, H0, H_ops, R, times, u_start=u0, eps=2 * pi / T, phase_sensitive=False,
                          progress_bar=TextProgressBar())

10.0%. Run time:  28.26s. Est. time left: 00:00:04:14
20.0%. Run time:  56.94s. Est. time left: 00:00:03:47
30.0%. Run time:  85.26s. Est. time left: 00:00:03:18
40.0%. Run time: 112.21s. Est. time left: 00:00:02:48
50.0%. Run time: 136.12s. Est. time left: 00:00:02:16
60.0%. Run time: 160.56s. Est. time left: 00:00:01:47
70.0%. Run time: 176.98s. Est. time left: 00:00:01:15
80.0%. Run time: 192.76s. Est. time left: 00:00:00:48
90.0%. Run time: 208.66s. Est. time left: 00:00:00:23
Total run time: 223.38s


In [ ]:
plot_grape_control_fields(times, result.u[:, :, :] / (2 * pi), H_labels, uniform_axes=True);
plt.show()